In [45]:
# pip install spotipy


[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [46]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json
import numpy as np
import pandas as pd

In [47]:
credentials = json.load(open('authorization.json'))
client_id = credentials['client_id']
client_secret = credentials['client_secret']

playlist_index = 0

playlist_uri = 'spotify:playlist:37i9dQZEVXbK4gjvS1FjPY' #top50Singapore
like = True

In [48]:
client_credentials_manager = SpotifyClientCredentials(client_id=client_id,client_secret=client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [49]:
uri = playlist_uri    # the URI is split by ':' to get the username and playlist ID
print(uri.split(':'))
username = uri.split(':')[1]
playlist_id = uri.split(':')[2]

['spotify', 'playlist', '37i9dQZEVXbK4gjvS1FjPY']


In [50]:
results = sp.user_playlist(username, playlist_id, 'tracks')

In [51]:
playlist_tracks_data = results['tracks']
playlist_tracks_data

{'href': 'https://api.spotify.com/v1/playlists/37i9dQZEVXbK4gjvS1FjPY/tracks?offset=0&limit=100&additional_types=track',
 'items': [{'added_at': '2023-03-17T12:08:53Z',
   'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/'},
    'href': 'https://api.spotify.com/v1/users/',
    'id': '',
    'type': 'user',
    'uri': 'spotify:user:'},
   'is_local': False,
   'primary_color': None,
   'track': {'album': {'album_group': 'single',
     'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6HvZYsbFfjnjFrWF950C9d'},
       'href': 'https://api.spotify.com/v1/artists/6HvZYsbFfjnjFrWF950C9d',
       'id': '6HvZYsbFfjnjFrWF950C9d',
       'name': 'NewJeans',
       'type': 'artist',
       'uri': 'spotify:artist:6HvZYsbFfjnjFrWF950C9d'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      '

In [52]:
playlist_tracks_data = results['tracks']
playlist_tracks_id = []
playlist_tracks_titles = []
playlist_tracks_artists = []
playlist_tracks_first_artists = []

for track in playlist_tracks_data['items']:
    playlist_tracks_id.append(track['track']['id'])
    playlist_tracks_titles.append(track['track']['name'])
    # adds a list of all artists involved in the song to the list of artists for the playlist
    artist_list = []
    for artist in track['track']['artists']:
        artist_list.append(artist['name'])
    playlist_tracks_artists.append(artist_list)
    playlist_tracks_first_artists.append(artist_list[0])

In [53]:
features = sp.audio_features(playlist_tracks_id)
features_df = pd.DataFrame(data=features, columns=features[0].keys())


In [54]:
features_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.804,0.771,9,-4.067,0,0.0433,0.3570,0.000003,0.1080,0.739,126.956,audio_features,65FftemJ1DbbZ45DUfHJXE,spotify:track:65FftemJ1DbbZ45DUfHJXE,https://api.spotify.com/v1/tracks/65FftemJ1Dbb...,https://api.spotify.com/v1/audio-analysis/65Ff...,212253,4
1,0.644,0.735,8,-5.747,1,0.0391,0.0521,0.144000,0.1610,0.418,88.980,audio_features,1Qrg8KqiBpW07V7PNxwwwL,spotify:track:1Qrg8KqiBpW07V7PNxwwwL,https://api.spotify.com/v1/tracks/1Qrg8KqiBpW0...,https://api.spotify.com/v1/audio-analysis/1Qrg...,153947,4
2,0.814,0.641,6,-5.957,0,0.1110,0.0270,0.000000,0.0993,0.183,133.854,audio_features,3r8RuvgbX9s7ammBn07D3W,spotify:track:3r8RuvgbX9s7ammBn07D3W,https://api.spotify.com/v1/tracks/3r8RuvgbX9s7...,https://api.spotify.com/v1/audio-analysis/3r8R...,185507,4
3,0.782,0.587,11,-8.300,0,0.0324,0.4280,0.000002,0.3370,0.760,120.041,audio_features,3Kw7zkALCVxY4wmlnh2IWC,spotify:track:3Kw7zkALCVxY4wmlnh2IWC,https://api.spotify.com/v1/tracks/3Kw7zkALCVxY...,https://api.spotify.com/v1/audio-analysis/3Kw7...,174253,4
4,0.575,0.500,1,-7.594,0,0.0707,0.2270,0.000000,0.3350,0.506,67.033,audio_features,4W4fNrZYkobj539TOWsLO2,spotify:track:4W4fNrZYkobj539TOWsLO2,https://api.spotify.com/v1/tracks/4W4fNrZYkobj...,https://api.spotify.com/v1/audio-analysis/4W4f...,232857,4


In [56]:
features_df['title'] = playlist_tracks_titles
features_df['first_artist'] = playlist_tracks_first_artists
features_df['all_artists'] = playlist_tracks_artists
#features_df = features_df.set_index('id')
features_df = features_df[['id', 'title', 'first_artist', 'all_artists',
                           'danceability', 'energy', 'key', 'loudness',
                           'mode', 'acousticness', 'instrumentalness',
                           'liveness', 'valence', 'tempo',
                           'duration_ms', 'time_signature']]
features_df.head()

,id,title,first_artist,all_artists,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,65FftemJ1DbbZ45DUfHJXE,OMG,NewJeans,[NewJeans],0.804,0.771,9,-4.067,0,0.3570,0.000003,0.1080,0.739,126.956,212253,4
1,1Qrg8KqiBpW07V7PNxwwwL,Kill Bill,SZA,[SZA],0.644,0.735,8,-5.747,1,0.0521,0.144000,0.1610,0.418,88.980,153947,4
2,3r8RuvgbX9s7ammBn07D3W,Ditto,NewJeans,[NewJeans],0.814,0.641,6,-5.957,0,0.0270,0.000000,0.0993,0.183,133.854,185507,4
3,3Kw7zkALCVxY4wmlnh2IWC,Cupid - Twin Ver.,FIFTY FIFTY,[FIFTY FIFTY],0.782,0.587,11,-8.300,0,0.4280,0.000002,0.3370,0.760,120.041,174253,4
4,4W4fNrZYkobj539TOWsLO2,Die For You (with Ariana Grande) - Remix,The Weeknd,"[The Weeknd, Ariana Grande]",0.575,0.500,1,-7.594,0,0.2270,0.000000,0.3350,0.506,67.033,232857,4
